In [ ]:
'''
Evaluate a Lanternfish autoencoder

Lanternfish uses autoencoders to learn representation of a motion feature
space in an unsupervised manner. 

This notebook outlines how to obtain autoencoder outputs for visualization.
'''
import keras.backend as K
from keras.models import load_model
from motcube_preprocessing import *
from scipy.io import savemat
import os

In [ ]:
# Set directories
model_path = '/path/to/model.h5'
input_dir = '/path/to/input/images'
save_dir = '/path/to/output'
# Set model parameters
exp_name = 'exp_name'
batch_size = 3
cube_size = (156,156,100)

In [ ]:
# Load model
model = load_model(model_path)

In [ ]:
# Define autoencoder data generator wrapper
def auto_gt_generator(generator):
    '''
    Wraps a generator to provide the same output twice in a tuple
    Useful for training with input as ground truth
    '''
    for batch in generator:
        yield (batch, batch)

In [ ]:
# Instantiate data generators
valgen = MotcubeDataGenerator()
val_generator = valgen.flow_from_directory(input_dir, class_mode = None, color_mode='grayscale', target_size=cube_size, batch_size = batch_size)
ae_gen = auto_gt_generator(val_generator)

In [ ]:
# Evaluate a single batch of images
x = next(val_generator)
evaluate_model = K.function(
    [model.layers[0].input, K.learning_phase()],
    [model.layers[-1].output]
    )
y = evaluate_model([x, 0])[0]

# y may be exported using scipy.io.savemat for visualization with matlab
savemat(os.path.join(save_dir, exp_name + '_ae_io.mat'), mdict = {'x' : x, 'y' : y})